In [1]:
from scipy import io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

filename = 'D:\\VR300Data\\0315\\Minchul\\Training\\mat\\Training-[2021.03.15-22.09.42].mat'

mat_file = io.loadmat(filename)

In [3]:
## DataProcessing and model generation process
import hdf5storage
import numpy as np
from scipy.signal import butter, lfilter
import os
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            
        return [Epochs, Num[0]]

def Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum):
        FeaturesT = np.zeros((NumT, featureNum))
        for i in range(NumT):
            FeaturesT[i,:] = np.reshape(EpochsT[i,:,:],(1,featureNum))
        FeaturesN = np.zeros((NumN, featureNum))
        for j in range(NumN):
            FeaturesN[j,:] = np.reshape(EpochsN[j,:,:],(1,featureNum))
        return [FeaturesT,FeaturesN]

def Balancing_DataSet(Epochs, size):
        Epochs_New = np.zeros((size, Epochs.shape[1], Epochs.shape[2]))
    
        index = np.random.choice(Epochs.shape[0], size = size, replace = False)
    
        Epochs_New = Epochs[index, :, :]
    
        return Epochs_New
    
def Standardization(Epochs):
        for i in range(Epochs.shape[1]):
            Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
            Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
        return Epochs

def Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, componentNum):
        EpochsT = Standardization(EpochsT)
        EpochsN = Standardization(EpochsN)
        NumT_Aver = NumT-componentNum
        NumN_Aver = NumN-componentNum
    
        EpochsT_Aver = np.zeros((NumT_Aver, channelNum, epochSampleNum))
        EpochsN_Aver = np.zeros((NumN_Aver, channelNum, epochSampleNum))
        for i in range(NumT_Aver):
            EpochsT_Aver[i, :, :] = np.mean(EpochsT[i:i+componentNum, :, :], axis=0)
        for j in range(NumN_Aver):
            EpochsN_Aver[j, :, :] = np.mean(EpochsN[j:j+componentNum, :, :], axis=0)
        
        return [EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver]

def plotGraph(filename):


        mat = hdf5storage.loadmat(filename)
        channelNames = mat['channelNames']
        eegData = mat['eegData']
        samplingFreq = mat['samplingFreq']
        samplingFreq = samplingFreq[0,0]
        stims = mat['stims']
        channelNum = channelNames.shape
        channelNum = channelNum[1]
        eegData = np.transpose(eegData)
        
        ##Preprocessing process

        #Bandpass Filter
        eegData = butter_bandpass_filter(eegData, 0.23, 30, samplingFreq, order=4)
    
        #Epoching
        epochSampleNum = int(np.floor(1.0 * samplingFreq))
        offset = int(np.floor(0.0 * samplingFreq)) 
        baseline = int(np.floor(1.0 * samplingFreq)) 
        [EpochsT, NumT] = Epoching(eegData, stims, 1, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        [EpochsN, NumN] = Epoching(eegData, stims, 0, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        
        EpochsN_New = Balancing_DataSet(EpochsN, NumT)
        
        #Convert to feature vector
        #[EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver] = Make_Average_Component(EpochsT, NumT, EpochsN_New, NumT, channelNum, epochSampleNum, 20)
        EpochsT = Standardization(EpochsT)
        EpochsN = Standardization(EpochsN)
        print(EpochsT.shape)
        print(EpochsN.shape)
        
        return [EpochsT, EpochsN]


def main():
    root = 'D:\\VR300Data\\0315\\Minchul\\Training\\mat\\Training-[2021.03.15-22.09.42].mat'
    T_all= np.zeros((210,7,300))
    N_all= np.zeros((1260,7,300))
    #for i in np.arange(1,56):
       # if(i<10):
      #      filename = root + '0' + str(i)
       # else:
       #     filename = root + str(i)
        #plotGraph(filename)
        #print("subject {0} is preprocessed".format(str(i)))
        
    [T_all[:,:,:],N_all[:,:,:]]=plotGraph(root)
    
    target_data = T_all
    non_target_data = N_all
    srate = np.squeeze(mat_file['samplingFreq'][0,0])
    target_data = target_data - np.mean(target_data, axis=0)
    non_target_data = non_target_data - np.mean(non_target_data, axis=0)
    
    baseline_target = np.mean(target_data[:, :, 0:int(srate * 3)], axis=2)
    baseline_non_target = np.mean(non_target_data[:, :, 0:int(srate * 0.3)], axis=2)

    target_data = target_data - np.expand_dims(baseline_target, axis=-1)
    non_target_data = non_target_data - np.expand_dims(baseline_non_target, axis=-1)
    target_data=target_data
    non_target_data=non_target_data

#-----------------------------------------
   # target_data = np.transpose(target_data, [1, 0, 2])
    #non_target_data = np.transpose(non_target_data, [1, 0, 2])
    output_file = "./resampled_VR300_0315_Minchul.npz"
    np.savez(output_file, np.array(target_data,dtype=float), np.array(non_target_data,dtype=float))
#-------------------------------------

    
if __name__ == "__main__":
    main()

(210, 7, 300)
(1260, 7, 300)
